In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Part 1
Flip-flop modules (prefix %) are either on or off; they are initially off. If a flip-flop module receives a high pulse, it is ignored and nothing happens. However, if a flip-flop module receives a low pulse, it flips between on and off. If it was off, it turns on and sends a high pulse. If it was on, it turns off and sends a low pulse.

Conjunction modules (prefix &) remember the type of the most recent pulse received from each of their connected input modules; they initially default to remembering a low pulse for each input. When a pulse is received, the conjunction module first updates its memory for that input. Then, if it remembers high pulses for all inputs, it sends a low pulse; otherwise, it sends a high pulse.

There is a single broadcast module (named broadcaster). When it receives a pulse, it sends the same pulse to all of its destination modules.

In [2]:
class Module:
    def __init__(self, input_line):
        input_line = input_line.strip()
        
        if '->' in input_line:
            name, targets = input_line.split(' -> ')
            if '%' in name:
                self.type = 'flip_flop'
                self.name = name[1:]
            elif '&' in name:
                self.type = 'conjunction'
                self.name = name[1:]
            else:
                self.type = ''
                self.name = name
            self.targets = [ t.strip() for t in targets.split(',') ]
        else:
            self.type = ''
            self.name = input_line.strip()
            self.targets = []
                
        self.mode = 0
        self.input_mods = {}
    
    def gets_signal(self, from_name, signal ):
        
        if from_name != '':
            self.input_mods[from_name] = signal
        
        if self.type == 'flip_flop':
            if signal == 0:
                if self.mode == 0:
                    self.mode = 1
                    return [ (self.name, dest, 1) for dest in self.targets]
                
                if self.mode == 1:
                    self.mode = 0
                    return [ (self.name, dest, 0) for dest in self.targets]
                
        elif self.type == 'conjunction':
            total_in = len(self.input_mods.keys())
            if sum(self.input_mods.values()) == total_in:
                send = 0
            else:
                send = 1
            return [ (self.name, dest, send) for dest in self.targets]
            
        else:
            return [ (self.name, dest, signal) for dest in self.targets]


In [3]:
from collections import deque

def hit_broadcast(module_dict, counts=[0,0]):
    counts[0] += 1
    pulses = deque(module_dict['broadcaster'].gets_signal('',0))
    idx = 0
    while len(pulses) > 0 :
        pulse = pulses.popleft()
        counts[pulse[2]] += 1
        new_pulses = module_dict[pulse[1]].gets_signal(pulse[0],pulse[2])
        if new_pulses is not None and len(new_pulses) > 0 :
            pulses.extend(new_pulses)
#        print (pulse)
    return counts

In [4]:
def do_part_one(file_path):
    modules = {}
    
    with open(file_path,'r') as f:
        for line in f.readlines():
            m = Module(line)
            modules[m.name] = m
    
    defined = list(modules.keys())
    moms = list(modules.values())
    for im in moms: 
        for t in im.targets:
            if t not in defined :
                modules[t] = Module(f'{t}')
                
            modules[t].input_mods[im.name] =0
    cnts=[0,0]
    for i in range(1000):
        cnts = hit_broadcast(modules,cnts)
        
    return cnts[0]*cnts[1]

In [5]:
%%time
do_part_one('input_data/test_20-0.txt')


CPU times: user 11.6 ms, sys: 1.41 ms, total: 13 ms
Wall time: 11.9 ms


11687500

In [6]:
%%time
do_part_one('input_data/day_20.txt')

CPU times: user 54.3 ms, sys: 2.13 ms, total: 56.5 ms
Wall time: 55.6 ms


763500168

# Part 2

In [37]:
from collections import deque

def hit_broadcast_pt2(module_dict):
    high_cnt ={ m : 0 for m in module_dict['dh'].input_mods.keys()}
    
    pulses = deque(module_dict['broadcaster'].gets_signal('',0))
    rx_low = 0
    
    while len(pulses) > 0 :
        pulse = pulses.popleft()
        new_pulses = module_dict[pulse[1]].gets_signal(pulse[0],pulse[2])
        if new_pulses is not None and len(new_pulses) > 0 :
            pulses.extend(new_pulses)
        if pulse[1] == 'dh' and pulse[2] == 1 :
            high_cnt[pulse[0]] +=1
    return high_cnt

In [50]:
from math import lcm
def do_part_two(file_path):
    modules = {}
    
    with open(file_path,'r') as f:
        for line in f.readlines():
            m = Module(line)
            modules[m.name] = m
    
    defined = list(modules.keys())
    moms = list(modules.values())
    for im in moms: 
        for t in im.targets:
            if t not in defined :
                modules[t] = Module(f'{t}')
                
            modules[t].input_mods[im.name] =0
    rnds = 0
    rx_low = 0
    
    
    high_sent = { m : [] for m in modules['dh'].input_mods.keys()}
    
    while rx_low == 0 :
        high_cnt = hit_broadcast_pt2(modules)
        rnds += 1
        for m,cnt in high_cnt.items():
            if cnt > 0 :
                high_sent[m].append(rnds)
        if sum([ len(x) > 0 for x in high_sent.values()]) == len(high_sent):
            break
    
    
    return lcm(*[ x[0] for x in high_sent.values()])

In [51]:
%%time
do_part_two('input_data/day_20.txt')

CPU times: user 124 ms, sys: 2.24 ms, total: 127 ms
Wall time: 127 ms


207652583562007

In [47]:
tmp

{'tr': [3739], 'xm': [3761], 'dr': [3797], 'nh': [3889]}